#Glyph-based Universe Visualization

####Team members: 
    Chris Jacobsen, Xin Tong, Ko-Chih Wang, Huijie Zhang

<img style="float: left" src="titleImage.png" /> <img style="float: center" src="titleImage2.png" />


##Outline

* <a href='#introduction'>Introduction</a>
* <a href='#dataloading'>Data Loading</a>
* <a href='#glyphbased'>Glyph-based Visualization</a>
* <a href='#interface'>Interface</a>
* <a href='#particleRendering'>Particle Rendering</a>
* <a href='#conclusions'>Conclusions and Future Work</a>
* <a href='#references'>References / Code Borrowed</a>


<a id='introduction'></a>
##Introduction

   Cosmological simulations produce massive data which is challenging to analyze and visualize. Identifying dark matter halos, which correspond to galaxies and clusters of galaxies, from a large number of particles is an active topic in cosmology. Earlier halo finding algorithms, such as FOF[Davis1985], use the 3-dimensional (3D) particle locations as data input. The 6DFOF algorithm [Maciejewski2009] improves the results by including particle velocity information as another 3 dimensions. The latest algorithm, Rockstar[Behroozi2011], incorporates time as one additional dimension (7D in total) to allow robust tracking of halos and again improve the halo finding results. Velocity has been an important factor in determining the particle-halo membership because two very close particles may have relative motion but may belong to different halos. The latest research [Behroozi2011] found out that the bulk velocity, which is the average of all halo particle velocities, is not a good representation of the halo velocity, because there is a difference between the velocities in the halo centers and the halo outskirts. Visualization of the velocity field for the particles in a halo or subhalo can help scientists to analyze the relative motion between nearby halos, discover the velocity difference among different regions of a halo, and verify the existing halo finding results.

   Current visualization mainly focuses on visualizing the locations of halos and the animation of halo locations over time but rarely visualizes its velocity. Woodring et al. [Woodring2011] uses oriented ellipsoids to visualize the velocities in ParaView software. An ellipsoid expresses the velocity direction withs its orientation and shows the velocity dispersion with color. This ellipsoid glyph has brief and limited uncertainty information and suffers from asymmetry and visual ambiguity as mentioned by Kindlmann[Kindlmann2004]. 
    
   In this project, we propose a distribution-based superquadric glyph technique to visualize the velocity field of the halo. We use surface color texture to show the histogram of the vector field within a halo and use the glyph shape to show the principal components of the distribution as an overview of the velocity statistics. This glyph gives more details of the velocity direction distribution of the particles within a halo than a simple velocity dispersion. We explore various ways to visualize the merge tree.  We construct a widget representing this tree, using it as a tool to select different time steps to view for the superquadric data, while locating and highlighting the corresponding selected halo (shown as a superquadric) in the scene. We also provide a merge tree visualization and glyph-based visualization system that allows users to navigate through the space interactively.


<a id='dataloading'></a>
##Data Loading

###Halo data loading

   The first step of this project is to understand the format of the cosmological data. The data set provided by VisConstest 2015 is an 11 GByte dataset. It includes 100 time steps of particle data and 89 time steps of halo information, and one file describes how the halos split and merge over time. Because our project uses superquadrics to visualize the velocities of particles belonging to a halo, the first step is to use the particle and halo data sets to identify the relation between halos and particles. However, the coordinate system between the particle and halo files are different. We use the following code to convert the particles into the halo coordinate system.
    
        if (i=0; i<n; i++) {
          printf("hello %d\n", i);
          x += 4;
        }
        
   The xx is the particle coordinate and what is what.
   We use the particle location, halo center and the halo radius to define whether a particle belongs to a halo. The radius of the halo must also be converted by the following code.
    
        if (i=0; i<n; i++) {
          printf("hello %d\n", i);
          x += 4;
        }
        
   After the conversion, we create the relation between the particles and halos. We also have the set of velocities of particles belonging to a halo. This is used to compute the shape and color texture of the glyphs and visualize the halo in the next section.
    
   In addition, we also collect the attributes (what halo it belongs to, velocity, halo radius and density) of the particles. This information is used to render the particle in the <a href='#particleRendering'>Particle Rendering</a> section. The collection process of velocity of particles, halo ID and radius have been described. Here, we introduce how we compute the density of a particle. The density of the particle is a useful piece of information to define the opacity or color of a particle when rendering the particle. For example, we can give a high density particle a high opacity because a high density region means there are more particles, and it could be a halo or will form a halo. The way to define the density of particles is to compute the number of particles in the local region. We divide the whole space into a 500x500x500 grid and compute the number of particles in a single cube to define the density of particles belonging to the cube. 
    
   After we finish the data loading and preprocessing step, we use the data structure and files to render the glyphs and particles without reloading the raw data and reprocessing. It makes the rendering efficient and the system interactive. 
   
### Merge Tree data loading

   The python flow reads the Dark Sky tree data file, working backwards in time from timestep 1.0 to the earliest time step for each listed merge tree.  For each halo id, it finds each of the halo records who cite the id as a descendant.  The original halo (at a later time step) becomes a parent node while the others become children in a tree structure.  Since each parent can have multiple children, it is analogous to a b-tree, though the tree stucture does not reorganize itself based on values.
   
   The structure of the tree is output to a text file in an in-order traversal ordering.  Counts are stored for the number of children for each parent in order to handle an abritrary number of children.  This ordering is then used to rebuild the tree by the C++ application.
   
<img src="MergeTreeDiagram.png" />

<a id='glyphbased'></a>
##Glyph-Based Visualization

###Color with Histogram
   One of the ways to express the value of bins is to use a colormap. Here a sphere can be used as the shape of the glyph, and we can map the bin values of the histogram onto the sphere with the colormap. If a certain area on the sphere has a red color, it means the vector directions from this area to the center of the sphere have high frequency on the histogram. The limitation of using color is occlusion. Half of the sphere surface is invisible unless we rotate the glyph. The sphere can be placed in different location of the volume with different sizes to represent the vector field distribution of different locations and different size of represented space. 

###Superquadric Shape with PCA
   Because of occlusion and visual clutter, it is not possible for users to perceive the entire histogram from the color texture. Thus, we use another two visual channels, shape and orientation, to represent the overall trend of the vector field distribution. 
   One simple option is to extrude the mesh of the previous mentioned sphere from the sphere center with the bin value as the amount of the extrusion. However, this will give us an arbitrary shape and may have a very rough surface when the histogram is noisy, which can introduce severe occlusion. Thus, we want to use a more interesting shape than a sphere but still keep its smoothness. Instead of encoding the entire information of the histogram, we only encode its principal directions by using the eigen vectors from principal component analysis (PCA). A good example of displaying the eigenvector is superquadric tensor glyph visualization[Kindlmann2004]. Here we also use a superquadric as the shape of our glyph to express the principal vector directions.

###Demonstration
<iframe width="853" height="480" src="https://www.youtube.com/embed/wC8WhLhtouw" frameborder="0" allowfullscreen></iframe>

<a id='interface'></a>
##Interface


###General User Interface

   Basic options for rotating and zooming are provided.  The user can rotate the scene by clicking and dragging the left mouse button and can zoom in and out using the scroll wheel on the mouse.  This allows the user to choose a view that works well for viewing the superquadric detail.

###Merge Tree Widget
   
   In our application, we provide a merge tree visualization widget.  This widget both serves the purpose of visualizing the merge tree structure and acting as a timestep selection tool.
   
   The component displays five timesteps starting from a starting step.  The "master tree" is cut off, beginning at this point, forming a series of subtrees in a forest.  The starting halo id for each of these is displayed in a dropdown box.  When the user selects one of these ids from the dropdown box, the subtree is then loaded into the merge tree widget,showing five timesteps.  The halo id for each node in the tree is shown, with the timestep marked in parentheses. 
   
   The user sees these different ids by expanding the nodes in the component.  As a node is expanded, the superquadric halo data is loaded into the scene if the data has been generated by the Python flow.  This allows the merge tree component to act as a timestep selection tool.  The corresponding halo is then highlighted with a yellow box in the scene.  Thus the widget also acts as a tool to hunt for halos from the merge tree across different time steps.
   
   The widget itself is based on JSon; we modified the JSon widget at (http://codereview.stackexchange.com/questions/11849/qjsonview-a-qwidget-based-json-explorer-for-qt) to serve our needs. Our application constructs a JSon representation of the data and feeds it to the widget.  The merge tree structure and halo structure records are both maintained by hash tables.  When a selection is made in the JSon widget, it finds the Merge Tree structure record by looking up the halo id in the hash table.  It is then able to find the halo in the scene by looking it up in the halo record hash table.
   
<img src="mergeTreeUserInterface.png" />

<a id='particleRendering'></a>
##Particle Rendering    
We consider one other possible way to visualize the trend of velocity in the space for scientists. We would like to visualize the velocity trend of the space by a relatively simple way -- plot the velocity of particles by different colors. The first step of this visualization is to decide the transfer function for this particle rendering. This is done in order to render the velocity of particles to make scientists perceive the velocity trend in the space. We decide to use the velocity of each particle to define the color. In our approach, for every velocity vector 
$v$, we normalize each element of velocity to $[0, 1]$ and get the new vector $v'$. We directly use the new vector $v'$ as color to the particle. To visualize this design, we can see the following figure.

<img src="colorcube.png" />

In the image, it illustrates the color mapping from the $v'$ to the color. The corner closest to the viewer is the vector $[0,0,0]$ and the corner farthest away from the view is $[1,1,1]$. We can see that the color gradually changes from the closest corner to the farthest corner. When we use this color map to plot the particle color, it shows the similar color if the particles have similar velocity. It helps the scientist to have a velocity overview over the space. For the opacity transfer function, we use the density attribute of the particle to define it. This makes the particle in the high density region more visible because usually the denser region is possibly a halo or interesting area. 

In the particle rendering, we also make the system be able to visualize the velocity of particles over time. In the animation of the demonstration, we can see that the overall space is divided into several big blocks by color. It means that the particles in the local region of the space usually move along a similar direction.

In the following demonstration, we also visualize the particles by other attributes. For example, we visualize the particle by the radius of the halo or the halo ID to which it belongs.

###Demonstration    
    
   Note: For the following demonstrations, we strongly recommend to use the setting to change the quality to the highest video resolution.
    
   Velocity of Particle Animation: We use the transfer function, which is described in the previous section to animate the velocity trend of the space. 

<iframe width="853" height="480" src="https://www.youtube.com/embed/WGzmbIcy4PI" frameborder="0" allowfullscreen></iframe>

   Halo Merge&Split Visualized by Particle Velocity: in this animation, zoom in to small part of the universe and you can see the process of halo merging and splitting.

<iframe width="853" height="480" src="https://www.youtube.com/embed/yntgi89q0vc" frameborder="0" allowfullscreen></iframe>

   Color by Halo ID: In this animation, we use the halo ID to color the particle. If a particle does not belong to any halo, we use purple to show the particle. The opacity of the particle is also defined by the density attribute of the particle.
    
<iframe width="853" height="480" src="https://www.youtube.com/embed/bi981qOOdxw" frameborder="0" allowfullscreen></iframe>

   Merge Tree Usage: In this video, we use the merge tree to view two different subtrees.  We load several different time steps as we browse the tree and see the selected halos become hilighted in the scene.
    
<iframe width="853" height="480" src="https://www.youtube.com/embed/_g77RlXv-nI" frameborder="0" allowfullscreen></iframe>

<a id='conclusions'></a>
##Conclusions and Future Work

   Although superquadrics require a bit of training for users, their ability to prevent occlusion and ability to summarize information with additional channels through their shape offer clear advantages.  Our integration of the tree widget to view data cross time steps with the view of super quadrics within one time step allows users to explore data and home in on items of interest.
   
   We are intending to combine the results of this project with an analysis of a separate data set, building a binary tree based on entropy splits.  It will be interesting to compare and contrast the super quadrics results between the two data sets in a research paper.  This also will give us opportunities to think about how an interactive tree widget can be used to explore a binary tree based on entropy.
   
   We also are interested in exploring additional tree visualization options.  We will probably give preference to the usage of the Graph Viz API, since this allows graphs to be generated for arbitrary graph structures.  We would use Graph Viz for the layout engine and then construct QT widgets based on the node positions.  We also are wanting to implement a tree map, which will allow a "recursive view" of the tree.
    

<a id='references'></a>
##References

    * [Davis1985] Marc Davis, George Efstathiou, Carlos S. Frenk and Simon D.M. White(1985) "The Evolution of Large Scale Structure in a Universe Dominated by Cold Dark Matter"
    * [Kindlmann2004] Gordon Kindlmann(2004) "Superquadric Tensor Glyphs"
    * [Maciejewski2009] M. Maciejewski, S. Colombi, C. Alard, F. Bouchet, and C. Pichon(2009) "Phase-space structures – I. A comparison of 6D density estimators"
    * [Behroozi2011] Peter S. Behroozi, Risa H. Wechsler and Hao-Yi Wu(2011) "The Rockstar Phase-Space Temporal Halo Finder and the Velocity Offsets of Cluster Cores"
    * [Woodring2011] Jonathan Woodring, Katrin Heitmann, James Ahrens, Patricia Fasel, Chung-Hsing Hsu, Salman Habib, Adrian Pope (2011) "Analyzing and Visualizing Cosmological Simulations with ParaView"
    * JSon Widget Original Code- http://codereview.stackexchange.com/questions/11849/qjsonview-a-qwidget-based-json-explorer-for-qt
    * Elastic Graph Visualization QT Widget (not currently used for the Dark Sky data but present in the code) - http://doc.qt.io/qt-5/qtwidgets-graphicsview-elasticnodes-example.html